# Génération d'objets combinatoire décrits par une grammaire

Ce notebook est utilisé pour executer les fonctions et méthodes de notre projet, ainsi qu'a des fins de tests.

In [1]:
from rules import UnionRule, ProductRule, SingletonRule, EpsilonRule, BoundRule, SequenceRule, init_grammar
from rules import Union, Prod, NonTerm, Singleton, Epsilon, expand_init_grammar


In [2]:
from binarytree import BinaryTree, Leaf

## Méthodes combinatoire

Les méthodes count, rank, list, unrank et random. Exemple avec les arbres binaires

In [3]:
from grammars import treeGram

treeGram["Tree"].list(5)

[(leaf, (leaf, (leaf, (leaf, leaf)))),
 (leaf, (leaf, ((leaf, leaf), leaf))),
 (leaf, ((leaf, leaf), (leaf, leaf))),
 (leaf, ((leaf, (leaf, leaf)), leaf)),
 (leaf, (((leaf, leaf), leaf), leaf)),
 ((leaf, leaf), (leaf, (leaf, leaf))),
 ((leaf, leaf), ((leaf, leaf), leaf)),
 ((leaf, (leaf, leaf)), (leaf, leaf)),
 (((leaf, leaf), leaf), (leaf, leaf)),
 ((leaf, (leaf, (leaf, leaf))), leaf),
 ((leaf, ((leaf, leaf), leaf)), leaf),
 (((leaf, leaf), (leaf, leaf)), leaf),
 (((leaf, (leaf, leaf)), leaf), leaf),
 ((((leaf, leaf), leaf), leaf), leaf)]

On doit avoir 14 arbres de taille 5

In [4]:
treeGram["Tree"].count(5)

14

Les implémentations de count et de list sont cohérentes entre elles

In [5]:
treeGram["Tree"].count(5) == len(treeGram["Tree"].list(5))

True

In [6]:
treeGram["Tree"].unrank(5,5)

((leaf, leaf), (leaf, (leaf, leaf)))

On doit obtenir la suite de nombres [0,1,2,3,4,5,6,7,8,9,10,11,12,13]

In [7]:
[ treeGram["Tree"].rank(o) for o in treeGram["Tree"].list(5) ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

Les implémentations de rank et de unrank sont cohérentes entre elles

In [8]:
tg=treeGram["Tree"]
all(tg.rank(tg.unrank(5,x)) == x for x in range(14))

True

La grammaire des mots de Dyck fonctionne comme attendu

In [9]:
from grammars import dyckGram
dg = dyckGram["Axiom"]
all(dg.rank(dg.unrank(6,x)) == x for x in range(dg.count(6)))

True

In [10]:
dg.count(8)

14

In [11]:
[ dg.rank(dyckWord) for dyckWord in dg.list(8) ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

## Grammaires expressives

On utilise la fonction expand_init_grammar afin de convertir une grammaire condensée en grammaire classique, cette fonction renvoie une nouvelle grammaire. L'initialisation avec init_grammar est faîte par cette fonction sur la grammaire résultante.

In [12]:
condensedTreeGram = {"Tree" : Union (Singleton(Leaf),Prod(NonTerm("Tree"), NonTerm("Tree"), "".join))}

In [13]:
expandedTreeGram = expand_init_grammar(condensedTreeGram)
expandedTreeGram

{'Prod_Tree_Tree': ProductRule(Tree,Tree),
 'Sing_leaf': SingletonRule(leaf),
 'Tree': UnionRule(Sing_leaf,Prod_Tree_Tree)}

Les deux grammaires sont bien les même

In [14]:
treeGram

{'Leaf': SingletonRule(leaf),
 'Node': ProductRule(Tree,Tree),
 'Tree': UnionRule(Node,Leaf)}

## Règle BoundRule

On peut utiliser la règle __BoundRule__ qui représente l'ensemble des éléments d'une grammaire d'une taille comprise entre lower_bound et upper_bound. Le paramètre n correspondant à la taille de l'ensemble est ignoré par cette règle.

In [15]:
from grammars import dyckGram

dg=dyckGram["Axiom"]
BoundRule(dg,0,8).list(0)

['',
 '()',
 '()()',
 '(())',
 '()()()',
 '()(())',
 '(())()',
 '(()())',
 '((()))',
 '()()()()',
 '()()(())',
 '()(())()',
 '()(()())',
 '()((()))',
 '(())()()',
 '(())(())',
 '(()())()',
 '((()))()',
 '(()()())',
 '(()(()))',
 '((())())',
 '((()()))',
 '(((())))']

## Règle SequenceRule

On peut utiliser la règle __SequenceRule__, celle-ci est implémentée comme héritère de __ConstructorRule__.

La syntaxe est SequenceRule(nonTerm,nonTermVide,cons)

In [16]:
sizestr = lambda w: len(w)
testSequence = {
    'letter_a' : SingletonRule('a'),
    'vide' : EpsilonRule(''),
    'sequence' : SequenceRule("letter_a", "vide", lambda a,b: a+b, lambda w: (w[0],w[1:]), sizestr)
}
init_grammar(testSequence)
BoundRule(testSequence["sequence"],0,10).list(0)

['',
 'a',
 'aa',
 'aaa',
 'aaaa',
 'aaaaa',
 'aaaaaa',
 'aaaaaaa',
 'aaaaaaaa',
 'aaaaaaaaa',
 'aaaaaaaaaa']

In [17]:
a=BoundRule(testSequence["sequence"],0,10)
[a.rank(el) for el in a.list(0)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [18]:
a.count(0)

11

In [19]:
a.unrank(5)

'aaaaa'

In [20]:
SeqGram = {
    "Vide" : EpsilonRule(''),
    "Seq"       : SequenceRule("Singleton", "Vide", lambda a,b: a+b, lambda string : (string[0], string[1:]), len),
    "Singleton" : UnionRule("AtomA", "AtomB", lambda string : 0 if string == "A" else 1, len),
    "AtomA"     : SingletonRule("A"),
    "AtomB"     : SingletonRule("B")
}
init_grammar(SeqGram)

"Valuation = {'Vide': 0, 'Seq': 0, 'Singleton': 1, 'AtomA': 1, 'AtomB': 1}"

In [21]:
SeqGram["Seq"].unrank(3,1)

'AAB'